In [15]:
from Tools import RSDC as RSDCsim
import numpy as np
import pandas as pd
from scipy.stats import norm

def create_transition_matrix(n_states, diagonal):
    """Create an N-dimensional transition matrix with a specified diagonal value."""
    matrix_1 = diagonal * np.eye(n_states)
    matrix_2 = (1 - diagonal) * (np.ones((n_states, n_states)) - np.eye(n_states)) / (n_states - 1)
    transition_matrix = matrix_1 + matrix_2
    return transition_matrix

def df_to_array(dataframe):
    """Convert a DataFrame to a NumPy array without transposing."""
    data_array = dataframe.to_numpy()  # No transpose here
    return data_array

def get_parameters(data, n_states=4):
    """Estimate parameters for each state based on the single time series data."""
    # Assuming the data is a 1D array
    overall_mean = np.mean(data)
    overall_std = np.std(data)
    
    params = np.zeros((n_states, 3))  # mu, phi (set to 0), sigma
    
    # Example strategy: distribute mu around the overall mean, and sigma as variations of overall std
    for state in range(n_states):
        params[state, 0] = overall_mean + np.random.uniform(-1, 1) * overall_std * 0.1  # mu
        params[state, 2] = overall_std * (1 + np.random.uniform(-0.1, 0.1))  # sigma
    
    return params

def get_densities(data, params):
    n_states = params.shape[0]
    T = len(data)
    # Initialize densities array
    densities = np.zeros((n_states, T))
    
    # Extract parameters for vectorized computation
    mu = params[:, 0]  # Shape (n_states,)
    sigma = params[:, 2]  # Shape (n_states,)
    
    # Ensure sigma > 0 to avoid division by zero
    sigma = np.clip(sigma, 1e-6, np.inf)
    
    # Reshape data to align for broadcasting
    # Data reshaped to (1, T) to broadcast over states
    X = data.reshape(1, T)  # Shape now (1, T)
    
    # Calculate densities using Gaussian PDF formula, vectorized across states
    # mu reshaped to (n_states, 1) and sigma reshaped to (n_states, 1) for broadcasting
    densities = (1. / (sigma[:, np.newaxis] * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((X - mu[:, np.newaxis]) / sigma[:, np.newaxis]) ** 2) + 1e-6
    
    return densities
def scaled_forward_backward(densities, transition_matrix):
    n_states, T = densities.shape
    alpha = np.zeros((n_states, T))
    beta = np.zeros((n_states, T))
    scale_factors = np.zeros(T)
    
    # Forward Pass
    alpha[:, 0] = densities[:, 0]
    scale_factors[0] = 1.0 / np.sum(alpha[:, 0])
    alpha[:, 0] *= scale_factors[0]
    
    for t in range(1, T):
        alpha[:, t] = np.dot(transition_matrix.T, alpha[:, t-1]) * densities[:, t]  # Using np.dot for clarity
        scale_factors[t] = 1.0 / np.sum(alpha[:, t])
        alpha[:, t] *= scale_factors[t]
    
    # Backward Pass
    beta[:, T-1] = scale_factors[T-1]
    
    for t in range(T-2, -1, -1):
        beta[:, t] = np.dot(transition_matrix, densities[:, t+1] * beta[:, t+1])
        beta[:, t] *= scale_factors[t]
    
    return alpha, beta, scale_factors

import numpy as np

def calculate_smoothed_probabilities(alpha, beta, transition_matrix, densities):
    T = alpha.shape[1]
    N = alpha.shape[0]
    
    # Smoothed state probabilities
    gamma = (alpha * beta) / np.sum(alpha * beta, axis=0)
    
    # Preallocate xi array
    xi = np.zeros((N, N, T-1))
    
    # Calculate xi for t in [0, T-2] (since it involves t and t+1)
    for t in range(T-1):
        xi_denominator = np.sum(alpha[:, t].reshape(-1, 1) * transition_matrix * densities[:, t+1] * beta[:, t+1], axis=(0, 1))
        xi[:, :, t] = (alpha[:, t].reshape(-1, 1) * transition_matrix * densities[:, t+1] * beta[:, t+1].reshape(1, -1)) / xi_denominator
    
    return gamma, xi
import numpy as np
def update_initial_and_transition_matrices(gamma, xi):
    """
    Update initial state probabilities and transition matrix based on smoothed probabilities.
    
    Parameters:
    - gamma: Smoothed state probabilities with shape (N, T), where N is the number of states and T is the number of time steps.
    - xi: Smoothed transition probabilities with shape (N, N, T-1).
    
    Returns:
    - initial_state_probabilities: Updated initial state probabilities based on gamma.
    - transition_matrix: Updated and normalized transition matrix based on xi.
    """
    N, T = gamma.shape
    
    # Initial state probabilities are simply the smoothed probabilities at t=0
    initial_state_probabilities = gamma[:, 0]
    
    # Sum xi across time to get cumulative transitions, then average by T-1 (number of transitions)
    transition_matrix = np.sum(xi, axis=2) / (T-1)
    
    # Normalize the rows of the transition matrix to sum to 1
    transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)
    
    return initial_state_probabilities, transition_matrix
import numpy as np

import numpy as np

def calculate_parameters(data, smoothed_states):
    N, T = smoothed_states.shape
    params = np.zeros((N, 3))  # Initializing for mu, phi, sigma
    
    if data.ndim > 1:
        data = data.flatten()  # Ensure data is 1D
    
    for state in range(N):
        weights = smoothed_states[state, :]
        
        if weights.ndim > 1 or data.ndim > 1:
            print("Unexpected array dimensions:", weights.ndim, data.ndim)
            continue
        
        # Calculate weighted mean (mu)
        weighted_sum = np.dot(weights, data)
        total_weight = weights.sum()
        mu = weighted_sum / total_weight if total_weight > 0 else 0

        # Initialize sums for phi calculation
        phi_numerator, phi_denominator = 0.0, 0.0
        for t in range(1, T):  # Start from 1 since we use x[t-1]
            x_t = data[t] - mu
            x_t_minus_1 = data[t-1] - mu
            phi_numerator += weights[t] * x_t * x_t_minus_1
            phi_denominator += weights[t] * x_t_minus_1**2
        
        # Calculate phi
        phi = phi_numerator / phi_denominator if phi_denominator > 0 else 0

        # Weighted calculation for sigma (standard deviation)
        weighted_variance = np.dot(weights, (data - mu)**2) / total_weight if total_weight > 0 else 0
        sigma = np.sqrt(weighted_variance) if weighted_variance > 0 else 0

        # Assign calculated parameters for the current state
        params[state, :] = mu, phi, sigma

    return params


def calculate_log_likelihood(scale_factors):
    """
    Calculate the log likelihood of the observed data given the model.
    
    Parameters:
    - scale_factors: Scaling factors from the forward pass, a 1D NumPy array of length T.
    
    Returns:
    - log_likelihood: The log likelihood of the observed data.
    """
    log_likelihood = np.sum(np.log(scale_factors))
    return log_likelihood


In [2]:
import time 
# Start timing
start_time = time.time()
# reload(EM)
# from EM import Base
O = 1000
K = 2
N = 2
# phi_vals = np.zeros((K,N))
sim = ARsim(n_states=N, K_series = K, num_obs = O, transition_diagonal=0.99, deterministic=True) #sigmas = [[0,0]]
sim.simulate()
df = sim.data
transition_guess=0.95
n_states=4
max_iterations=100 
tolerance=1e-5
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")

Elapsed time: 0.05765 seconds


In [3]:
# data = df_to_array(df)  # Step 1
# transition_matrix = create_transition_matrix(n_states, transition_guess)  # Provided Step 2
# print(transition_matrix)
# parameters = get_parameters(data, n_states)  # Step 4
# densities = get_densities(data, parameters)  # Step 5
# alpha, beta, scale_factors = scaled_forward_backward(densities, transition_matrix)  # Step 6


In [4]:
import time

# Start timing
start_time = time.time()

# Place the code you want to time here
# Example dummy operation: a loop that runs for a bit

data = df_to_array(df)  # Step 1
transition_matrix = create_transition_matrix(n_states, transition_guess)  # Provided Step 2
# print(transition_matrix)
parameters = get_parameters(data, n_states)  # Step 4

densities = get_densities(data, parameters)  # Step 5
alpha, beta, scale_factors = scaled_forward_backward(densities, transition_matrix)  # Step 6
smoothed_states, smoothed_transitions = calculate_smoothed_probabilities(alpha, beta, transition_matrix, densities)
initial_state_probabilities, updated_transition_matrix = update_initial_and_transition_matrices(smoothed_states, smoothed_transitions )
param_2 = calculate_parameters(data, smoothed_states)
ll = calculate_log_likelihood(scale_factors)
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")
print(parameters)
print(param_2)

Elapsed time: 0.06178 seconds
[[ 0.0303717   0.          5.35243681]
 [-0.49561526  0.          5.16528636]
 [-0.22051072  0.          4.94637482]
 [-0.66507866  0.          5.26669214]]
[[ 0.06536125 -0.1793779   5.23445705]
 [-0.41450497 -0.12723425  5.05021325]
 [-0.10282253 -0.07381964  4.78113331]
 [-0.5936103  -0.14641698  5.16793715]]


In [5]:
# sim.mu

array([[-0.09720349,  0.03333938, -0.04448732,  0.09408589]])

In [6]:
# sim.phi

array([[-0.06295246, -0.66076108,  0.2766486 , -0.39040169]])

In [7]:
# ll

3039.454334501066

In [8]:
# sim.sigmas

array([[5.67787423, 0.47807888, 4.3887414 , 5.014632  ]])

In [9]:
# np.sum(updated_transition_matrix, axis=1)

array([1., 1., 1., 1.])

# RSDC


In [20]:
import numpy as np

def calculate_garch_std(data, omega, alpha, beta):
    """
    Calculate the conditional standard deviation for each time point in a time series
    using the GARCH(1,1) model parameters.

    Parameters:
    - data: 1D NumPy array of time series data.
    - omega, alpha, beta: GARCH(1,1) model parameters.

    Returns:
    - std_series: 1D NumPy array of conditional standard deviations.
    """
    T = len(data)
    std_series = np.zeros(T)
    std_series[0] = np.sqrt(omega / (1 - alpha - beta))  # Initial standard deviation
    
    for t in range(1, T):
        std_series[t] = np.sqrt(omega + alpha * (data[t-1] - 0)**2 + beta * (std_series[t-1])**2)
    
    return std_series
def calculate_standardized_residuals(data, std_series):
    """
    Calculate the standardized residuals for a time series given
    the conditional standard deviations from a GARCH(1,1) model.

    Parameters:
    - data: 1D NumPy array of time series data.
    - std_series: 1D NumPy array of conditional standard deviations.

    Returns:
    - standardized_residuals: 1D NumPy array of standardized residuals.
    """
    standardized_residuals = data / std_series
    return standardized_residuals
import numpy as np

# def calculate_state_correlations(standardized_residuals, smoothed_probabilities):
#     """
#     Calculate state-specific correlation matrices.

#     Parameters:
#     - standardized_residuals: 2D NumPy array (T x M) of standardized residuals, where T is the number of time points and M is the number of series.
#     - smoothed_probabilities: 2D NumPy array (N x T) of smoothed state probabilities, where N is the number of states.

#     Returns:
#     - correlation_matrices: 3D NumPy array (N x M x M) of correlation matrices for each state.
#     """
#     N, T = smoothed_probabilities.shape
#     _, M = standardized_residuals.shape
#     correlation_matrices = np.zeros((N, M, M))

#     for n in range(N):
#         # Initialize sum of outer products and sum of probabilities for normalization
#         sum_outer_products = np.zeros((M, M))
#         sum_probabilities = 0.0
        
#         for t in range(T):
#             prob = smoothed_probabilities[n, t]  # Probability of state n at time t
#             residuals = standardized_residuals[t, :].reshape(M, 1)  # Column vector of residuals at time t
#             outer_product = residuals @ residuals.T  # Outer product
            
#             # Weighted sum of outer products
#             sum_outer_products += prob * outer_product
#             # Sum of probabilities for normalization
#             sum_probabilities += prob
        
#         # Normalize to get the correlation matrix for state n
#         correlation_matrices[n] = sum_outer_products / sum_probabilities if sum_probabilities > 0 else np.zeros((M, M))
        
#         # Optionally, adjust the diagonal to 1 if the correlation matrix definition requires it
#         np.fill_diagonal(correlation_matrices[n], 1)

#     return correlation_matrices

# import numpy as np

def calculate_state_correlations(standardized_residuals, smoothed_probabilities):

    N, T = smoothed_probabilities.shape  # N states, T time points
    K = standardized_residuals.shape[1]  # K time series

    # Initialize matrices to accumulate results
    weighted_outer_products = np.zeros((N, K, K))
    sum_probabilities = np.zeros((N, 1, 1))

    for t in range(T):
        # Residuals at time t: shape (K, 1)
        u_t = standardized_residuals[t, :].reshape((K, 1))
        # Outer product at time t: shape (K, K)
        outer_product_t = u_t @ u_t.T
        
        # Update weighted sums and probabilities
        for n in range(N):
            weight = smoothed_probabilities[n, t]
            weighted_outer_products[n] += weight * outer_product_t
            sum_probabilities[n] += weight

    # Normalize to compute correlation matrices: shape (N, K, K)
    correlation_matrices = np.array([weighted_outer_products[n] / sum_probabilities[n]
                                     if sum_probabilities[n] > 0 else np.eye(K)
                                     for n in range(N)])

    # Ensure diagonal elements are 1
    for n in range(N):
        np.fill_diagonal(correlation_matrices[n], 1)

    return correlation_matrices


# Example parameters
omega = 0.0001
alpha = 0.05
beta = 0.94

# Calculate conditional standard deviations
std_series = calculate_garch_std(data, omega, alpha, beta)

# Calculate standardized residuals
standardized_residuals = calculate_standardized_residuals(data, std_series)



/tmp/ipykernel_60707/3332288789.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  std_series[t] = np.sqrt(omega + alpha * (data[t-1] - 0)**2 + beta * (std_series[t-1])**2)


In [17]:
np.min(standardized_residuals)

-159.34661229030573

In [21]:
R = calculate_state_correlations(standardized_residuals, smoothed_states)

In [22]:
R.shape

(4, 1000, 1000)